# MKV-SDE with polynomial drift 

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import time
from numpy import linalg as LA
from numpy import mean
from tabulate import tabulate
import mlflow
from itertools import product

from QUADRATIC import monte_carlo, base, euler, stochastic_gradient_descent_plot, stochastic_gradient_descent_table, set_experiment, run

## Plots

### $T=0.1$, $T=0.5$ and $T=1$

In [ ]:
delta = 0.8
X0 = 1
M1 = 10**7
basis_type = 'lagrange'
eps = 0.01

T = [0.1, 0.5, 1]
N = [10, 50, 100]
N1 = [10, 50, 100]
h = 0.01

n = 3 
M = 1000
r0 = [10, 5, 1]
rho = [0.6, 0.6, 0.6]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes = axes.flatten()

for i in range(3):

    # Euler - Monte Carlo

    start = time.process_time()
    X, gamma1, gamma2 = monte_carlo(delta, T[i], N1[i], M1, X0)
    end = time.process_time()

    print("Euler - Monte Carlo execution time: ", end - start)
    print(" ")

    # Stochastic Gradient Descent
    a1_0, a2_0, g = base(T[i], N[i], n, X0, basis_type)

    start = time.process_time() 
    a1, a2, m = stochastic_gradient_descent_plot(a1_0, a2_0, n, r0[i], rho[i], delta, N[i], M, X0, eps, h, g, gamma1, gamma2)
    end = time.process_time()

    print("SGD execution time: ", end - start)
    print("SGD steps: ", m)
    print(" ")

    ax = axes[i]
    ax.set_xlabel("Time")
    ax.plot(np.linspace(0, T[i], N[i]+1), np.dot(a1, g), '--', label='$\mathcal{L}\mathbf{a}_m^1$')
    ax.plot(np.linspace(0, T[i], N[i]+1), gamma1, label='$\overline{\gamma}^{MC}_1$')
    ax.plot(np.linspace(0, T[i], N[i]+1), np.dot(a2, g), '-.', label='$\mathcal{L}\mathbf{a}_m^2$')
    ax.plot(np.linspace(0, T[i], N[i]+1), gamma2, label='$\overline{\gamma}^{MC}_2$')
    ax.legend()
    ax.set_title(f"T={T[i]}")

plt.tight_layout()
# plt.savefig("QUAD_all_T.pdf")
plt.show()

## Tables

### $T=0.1$, $T=0.5$ and $T=1$

In [ ]:
delta = 0.8
X0 = 1
M1 = 10000000
basis_type = 'lagrange'
eps = 0.01

T_list = [0.1, 0.5, 1]  
N_list = [10, 50, 100]
N1_list = [10, 50, 100]
h = 0.01
n = 3
M_list = [1, 10, 100, 1000]
r0_list = [ [10, 10, 10, 10, 1],
            [1, 5, 5, 5],
            [1, 1, 1, 1]]
rho_list = [ [0.6, 0.6, 0.6, 0.6],
                [0.6, 0.7, 0.6, 0.6],
                [0.6, 0.6, 0.6, 0.6]]

eps = 0.01
repetition = 1000

with open("Tables QUAD with 1000 repetitions.txt", "w") as f:

    m1 = np.zeros((len(T_list), len(M_list)))
    m2 = np.zeros((len(T_list), len(M_list)))
    m3 = np.zeros((len(T_list), len(M_list)))

    for i, T in enumerate(T_list):

        start = time.process_time()   # the stopwatch starts
        X, gamma1, gamma2 = monte_carlo(delta, T, N1_list[i], M1, X0)
        end = time.process_time()   # the stopwatch stops
        f.write("Euler - Monte Carlo execution time: "+str(end - start))
        f.write("\n")
        f.write("\n")
        f.write("Number of iterations to achieve convergence with T = "+str(T)+" :")
        f.write("\n")
        f.write("\n")

        a1_0, a2_0, g = base(T, N_list[i], n, X0, basis_type)

        for j, M in enumerate(M_list):
            r0, rho = r0_list[i][j], rho_list[i][j]

            start = time.process_time()   # the stopwatch starts
            mm = [stochastic_gradient_descent_table(a1_0, a2_0, n,r0, rho, delta, N_list[i], M, X0, eps, h, g, gamma1, gamma2) for _ in range(repetition)]
            end = time.process_time()   # the stopwatch stops
            m1[i, j] = mean(mm)
            m2[i, j] = min(mm)
            m3[i, j] = max(mm)

            f.write(f"Time with n={n}, M={M}, r0={r0}, rho={rho}: {(end - start)/repetition}")
            f.write("\n")

            print("done n = "+str(M))
        print("done T = "+str(T))

    headerslist = [f"M={M}" for M in M_list]
    lateral_headers =[f"n={T}"for T in T_list]

    f.write("\n")
    f.write("\n")

    f.write(f"Average")
    f.write("\n")
    f.write(tabulate(np.column_stack((lateral_headers, m1)), headers=headerslist, colalign=("right",)))
    f.write("\n")
    f.write("\n")
    
    f.write(f"Min")
    f.write("\n")
    f.write(tabulate(np.column_stack((lateral_headers, m2)), headers=headerslist, colalign=("right",)))
    f.write("\n")
    f.write("\n")
    
    f.write(f"Max")
    f.write("\n")
    f.write(tabulate(np.column_stack((lateral_headers, m3)), headers=headerslist, colalign=("right",)))
    f.write("\n")
    f.write("\n")

### Mlflow procedure

In [ ]:
experiment_name = "T=1" 
parent_run_name = " "

shared_params = {
    'T': 1, 'n':3, 
    'N':100, 'N1':100, 'delta':0.8, 'X0':1, 'M1':10**8, 'h':0.01, 
    'eps':0.01, 'repetition':1000, 'basis_type':'lagrange',
    }

explored_params = {
    'rho': [0.6, 0.7],
    'M': [1, 10, 100, 1000]
}

def main():

    experiment_id = set_experiment(experiment_name)

    args_list = [] 

    for vals in product(*list(explored_params.values())):
        params_subset = {key:val for key, val in zip(list(explored_params.keys()), vals)}
        params_arg = {**shared_params, **params_subset}
        run_name = '-'.join([f'{k}:{v}' for k, v in params_subset.items()])
        args = [run_name, experiment_name, params_arg]
        args_list.append(args)

    with mlflow.start_run(experiment_id=experiment_id, run_name=parent_run_name): # parent run       

        mlflow.log_params(shared_params)

        results = []
        for args in args_list:
            results.append(run(*args))

if __name__ == "__main__":
    main()